In [ ]:
object std{
    type Function1[A, B] = A => B
    type Reader[S, A] = S => A
    type State[S, A] = S => (S, A) // ~= S => S & S => A
    type StateT[F[_], S, A] = S => F[(S, A)]
    type ReaderT[F[_], S, A] = S => F[A]
}

In [6]:
case class User(name: String, age: Int)

defined class User

In [ ]:
def validateName(name: String): String = if (name.length < 5) throw new Exception("") else name

In [ ]:
def validateAge(age: Int): Int = if (age > 55) throw new Exception("") else age

In [ ]:
def validateUser(name: String, age: Int): User = {
    val nameV: String = validateName(name) ; 
    val ageV: Int = validateAge(age) ; 
    return User(nameV, ageV)
}
    

In [16]:
def validateUser(name: String, age: Int): User = 
    User.apply(validateName(name): String, validateAge(age): Int)

cmd16.sc:2: polymorphic expression cannot be instantiated to expected type;
 found   : [F[_]]F[String]
 required: String
    User(validateName(name), validateAge(age))
                     ^cmd16.sc:2: polymorphic expression cannot be instantiated to expected type;
 found   : [F[_]]F[Int]
 required: Int
    User(validateName(name), validateAge(age))
                                        ^Compilation Failed

: 

In [ ]:
def validateName(name: String): Option[String] = if (name.length < 5) None else Some(name)

In [ ]:
def validateAge(age: Int): Option[Int] = if (age > 55) None else Some(age)

In [ ]:
def validateUser(name: String, age: Int): Option[User] = for {
    nameV <- validateName(name)
    ageV <- validateAge(age)
} yield User(nameV, ageV)
    

In [10]:
def validateName[F[_]](name: String)(implicit AE: ApplicativeError[F, String]): F[String] = 
    if (name.length < 5) "wrong name".raiseError[F, String] else name.pure[F]

defined function validateName

In [11]:
def validateAge[F[_]](age: Int)(implicit AE: ApplicativeError[F, String]): F[Int] = 
    if (age > 55) "wrong age".raiseError[F, Int] else age.pure[F]

defined function validateAge

In [12]:
def validateUser[F[_]](name: String, age: Int)(implicit AE: MonadError[F, String]): F[User] = for {
    nameV <- validateName[F](name)
    ageV <- validateAge[F](age)
} yield User(nameV, ageV)
    

defined function validateUser

In [16]:
def validateUser[F[_]](name: String, age: Int)(implicit AE: MonadError[F, String]): F[User] =
    validateName[F](name).flatMap{ nameV => 
        validateAge[F](age).flatMap{ ageV => 
            User(nameV, ageV).pure[F]
        }
    }


defined function validateUser

In [15]:
validateUser[Either[String, ?]]("dsfsdfsf", 60)

res14: Either[String, User] = Left("wrong age")

In [18]:
def validateUser[F[_]](name: String, age: Int)(implicit AE: ApplicativeError[F, String]): F[User] =
    (validateName[F](name), validateAge[F](age)).mapN(User.apply)


defined function validateUser

In [19]:
type Id[T] = T
type Cons[A, T] = A

defined type Cons

In [21]:
def constFlatMap[A] = new FlatMap[Cons[A, ?]]{
    //def flatMap[T1, T2](fa: Cons[A, T1])(cont: T1 => Cons[A, T2]): Cons[A, T2] = ???
    //def tailRecM[T1, T2](a: T1)(f: T1 => Cons[A, Either[T1, T2]]): Cons[A, T2] = ???
    //def map[T1, T2](fa: Cons[A, T1])(f: T1 => T2): Cons[A, T2] = ???
    def flatMap[T1, T2](fa: A)(cont: T1 => A): A = ???
    def tailRecM[T1, T2](a: T1)(f: T1 => A): A = ???
    def map[T1, T2](fa: A)(f: T1 => T2): A = ???
    
}

defined function constFlatMap

In [24]:
def constApplicative[A: Monoid]: Applicative[Cons[A, ?]] = new Applicative[Cons[A, ?]]{
    def pure[T1](a: T1): Cons[A, T1] = Monoid[A].empty : A
    def ap[T1, T2](f: Cons[A, T1 => T2])(a: Cons[A, T1]): Cons[A, T2] = Monoid[A].combine(f, a) : A
}

defined function constApplicative

In [ ]:
def myap[F[_]: Monad, T1, T2](fa: F[T1 => T2])(a: F[T1]): F[T2] = ???